# Revenu médian par catégorie Sociopro

In [12]:
import pandas as pd
import os

file_path = "../data_brute/disparités h-f.xlsx"
xls = pd.ExcelFile(file_path)
df = pd.read_excel(xls, sheet_name="Figure 2")


In [13]:
df.head()

,Figure 2 – Écarts de salaire et de temps de travail moyens entre les femmes et les hommes dans le secteur privé en 2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Caractéristiques,Salaire mensuel net en EQTP\n(en euros),NaN,NaN,Volume de travail \n(en EQTP),NaN,NaN
2,NaN,Femmes,Hommes,Écart (en %),Femmes,Hommes,Écart (en %)
3,Catégorie socioprofessionnelle,NaN,NaN,NaN,NaN,NaN,NaN
4,Cadres1,4120,4849,15,0.81,0.85,4.4
